In [ ]:
!pip install cirq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [ ]:
import cirq
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from scipy.linalg import eigh

In [ ]:
def load_and_scale_iris():
    """
    アイリスデータセットを読み込み、標準化して返します。
    """
    data = load_iris()
    iris_df = pd.DataFrame(data.data, columns=data.feature_names)
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(iris_df)
    return scaled_features  # 全てのサンプルを使用

In [ ]:
def create_quantum_encoding_circuit(qubits, data):
    """
    データを量子状態にエンコードするための回路を作成します。
    """
    circuit = cirq.Circuit()
    for i, value in enumerate(data):
        # 振幅エンコーディングとしてRyゲートを使用
        # 値の範囲を -1 から 1 の間にクリップして arccos に適用できるようにする
        value = np.clip(value, -1.0, 1.0)
        angle = 2 * np.arccos(value)
        if i < len(qubits):
            circuit.append(cirq.ry(angle)(qubits[i]))
    return circuit

In [ ]:
def perform_qpca(qubits, circuits, num_iterations):
    """
    qPCAを実施し、エンコードされた量子データに対して次元削減を行います。
    """
    simulator = cirq.Simulator()
    density_matrices = []
    for circuit in circuits:
        state = simulator.simulate(circuit)
        density_matrix = np.outer(state.final_state_vector, np.conj(state.final_state_vector))
        density_matrices.append(density_matrix)

    # 全ての密度行列を平均して最終的な密度行列を作成
    avg_density_matrix = np.mean(density_matrices, axis=0)

    # 固有値と固有ベクトルを計算（qPCAの中心的な部分）
    eigenvalues, eigenvectors = eigh(avg_density_matrix)

    print("Eigenvalues:")
    print(eigenvalues)
    print("Eigenvectors:")
    print(eigenvectors)

    # 固有値が大きいものに基づいて次元削減
    sorted_indices = np.argsort(eigenvalues)[::-1]
    top_eigenvectors = eigenvectors[:, sorted_indices[:num_iterations]]

    return top_eigenvectors, eigenvalues, sorted_indices

In [ ]:
def decode_quantum_data(top_eigenvectors, scaled_data):
    """
    qPCAで次元削減した量子データを古典データに低次元空間に射影します
    """
    # 元のデータを低次元空間に射影するために、データと固有ベクトルの次元を整合させる
    top_eigenvectors_reduced = top_eigenvectors[:scaled_data.shape[1], :]
    reduced_data = np.dot(scaled_data, top_eigenvectors_reduced)
    return reduced_data

In [ ]:
def calculate_contribution_ratios(eigenvalues, sorted_indices, num_components):
    """
    寄与率を計算し、各主成分の寄与率の表を作成します。
    """
    total_variance = np.sum(eigenvalues)
    selected_eigenvalues = eigenvalues[sorted_indices[:num_components]]
    contribution_ratios = selected_eigenvalues / total_variance
    contribution_table = pd.DataFrame({
        'Component': [f'PC{i+1}' for i in range(num_components)],
        'Eigenvalue': selected_eigenvalues,
        'Contribution Ratio': contribution_ratios
    })
    return contribution_table

In [ ]:
def main():
    # アイリスデータセットの読み込みと標準化
    scaled_data = load_and_scale_iris()

    # 量子ビットの準備
    num_qubits = scaled_data.shape[1]
    qubits = [cirq.LineQubit(i) for i in range(num_qubits)]

    # 各サンプルを量子エンコードし、回路を作成
    circuits = [create_quantum_encoding_circuit(qubits, data) for data in scaled_data]

    # 作成した各回路を表示
    for i, circuit in enumerate(circuits):
        print(f"Quantum Encoding Circuit for sample {i}:")
        print(circuit)

    # qPCAの実行
    top_eigenvectors, eigenvalues, sorted_indices = perform_qpca(qubits, circuits, num_iterations=2)
    print("Top Eigenvectors after qPCA:")
    print(top_eigenvectors)

    # qPCAで次元削減したデータをデコードして古典データに戻す
    decoded_data = decode_quantum_data(top_eigenvectors, scaled_data)
    print("Decoded Classical Data:")
    print(decoded_data)

    # 寄与率の計算と表の作成
    contribution_table = calculate_contribution_ratios(eigenvalues, sorted_indices, num_components=2)
    print("Contribution Ratios Table:")
    print(contribution_table)

In [ ]:
if __name__ == "__main__":
    main()

Quantum Encoding Circuit for sample 0:
0: ───Ry(1.71π)───

1: ───Ry(0)───────

2: ───Ry(2π)──────

3: ───Ry(2π)──────
Quantum Encoding Circuit for sample 1:
0: ───Ry(2π)──────

1: ───Ry(1.08π)───

2: ───Ry(2π)──────

3: ───Ry(2π)──────
Quantum Encoding Circuit for sample 2:
0: ───Ry(2π)───────

1: ───Ry(0.787π)───

2: ───Ry(2π)───────

3: ───Ry(2π)───────
Quantum Encoding Circuit for sample 3:
0: ───Ry(2π)───────

1: ───Ry(0.937π)───

2: ───Ry(2π)───────

3: ───Ry(2π)───────
Quantum Encoding Circuit for sample 4:
0: ───Ry(2π)───

1: ───Ry(0)────

2: ───Ry(2π)───

3: ───Ry(2π)───
Quantum Encoding Circuit for sample 5:
0: ───Ry(1.36π)───

1: ───Ry(0)───────

2: ───Ry(2π)──────

3: ───Ry(2π)──────
Quantum Encoding Circuit for sample 6:
0: ───Ry(2π)───────

1: ───Ry(0.421π)───

2: ───Ry(2π)───────

3: ───Ry(2π)───────
Quantum Encoding Circuit for sample 7:
0: ───Ry(2π)───────

1: ───Ry(0.421π)───

2: ───Ry(2π)───────

3: ───Ry(2π)───────
Quantum Encoding Circuit for sample 8:
0: ───Ry(2π)─